In [ ]:
!pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from pandas_datareader.data import DataReader 
plt.style.use("seaborn")
pd.options.display.float_format = '{:.4f}'.format


In [ ]:
symbol = "AAPL"

df = yf.download(tickers = symbol)
print(df)
df.index
df.info()

Pice Chart 

In [ ]:
df.Close.plot(figsize = (12,8), fontsize =12)
plt.ylabel("Price (USD)")
plt.title("AAPL Price Chart", fontsize = 15)
plt.show()

Volume Chart 

In [ ]:
df.loc["2022-6", "Volume"].plot(figsize = (12,8), kind = "bar")
plt.ylabel("Volume (Shares)")
plt.title("AAPL Trading Volume (daily)", fontsize = 15)
plt.show()

In [ ]:
df.Volume.mul(df.Close) # Trading Volume (USD Value)

Dividend

In [ ]:
df = yf.download(tickers = symbol, actions = True)
df.loc[df.Dividends != 0]
df.Dividends.sum() # cumulative Dividends per share
df.Close[-1] - df.Close[0] # cumulative Stock Price Increase
df.loc[df.Dividends != 0].Dividends.plot()
plt.show()

Adjusted Close price

In [ ]:
df2 = df[["Close", "Adj Close", "Dividends"]].copy()
print(df2)

df2.loc["2015":].plot(figsize = (12,8), secondary_y = "Dividends")
plt.show()

df2[df2.Dividends > 0]

last_div = df2.loc["2022-08-01" : "2022-08-5"].copy()
print(last_div)

print(last_div.Close - last_div["Adj Close"])

multiplier = (1 - 0.23/165.809998)
print(multiplier)

last_div["Adj Close calc"] = last_div.Close * multiplier
print(last_div)

print(df["Adj Close"] / df["Close"]) # Total Multiplier( All dividends)



Stock Splits

In [ ]:
splits = df.loc[df["Stock Splits"] > 0]
print(splits)

df["Stock Splits"].replace(to_replace = 0, value = 1, inplace = True)
print(df)

df["Multiplier"] = df.sort_index(ascending = False)["Stock Splits"].cumprod() # cumlative mulitplier
print(df)

df["Close_unadj"] = df["Close"] * df.Multiplier # obersvable close price
print(df)

df[["Close", "Close_unadj"]].plot(figsize = (12,8))
plt.vlines(splits.index, 0, 700, color = "black")
plt.show()


unadjusted Dividends

In [ ]:
df["div_unadj"] = df["Dividends"] * df.Multiplier
df.loc[df.div_unadj != 0].div_unadj.plot(figsize = (12,8))
plt.show()

Unadjusted Trading Volume (number of shares)

In [ ]:
df["Volume_unadj"] = df["Volume"] / df.Multiplier
print(df)

df["Volume_unadj"].plot(figsize = (12,8))
plt.show()

Stocsk from other Countries / Exchanges List

In [ ]:
yf.download(tickers = "APPL") # US Ticker(no suffix)
yf.download(tickers = "RELIANCE") # Indain Ticker(no suffix)
yf.download(tickers = "RELIANCE.NS") # Indian Ticker
yf.download(tickers = "LHA.DE")  # German Ticker(Deutsche Boerse XETRA)
yf.download(tickers = "LHA.F") # German Ticker (Frankfurt Stock Exchange)


Multiplier Tickers

In [ ]:
df = yf.download(tickers = ["APPL","MSFT"])
print(df)

print(df.info())
print(df.columns)
print(df.Close)

df.loc[:, ("Close", "MSFT")] # one specific column
df.Close.MSFT # one specific column

df.loc[:, (slice(None), slice("APPl"))] # inner index level
print(df)

df = df.swaplevel(axis = "columns").sort_index(axis = "columns")
print(df)

print(df.AAPL)

df = yf.download(tickers = ["LHA.F", "LHA.DE"]).Close
print(df)


Saving and Loading Data (Local Files)

In [ ]:
symbol = "AAPL"
print(df.to_csv(path_or_buf = "apple.csv")) # writing to local csv

df2 = pd.read_csv("apple.csv", index_col = "Date", parse_dates = ["Date"]) # laoding from local csv
print(df2)
print(df2.info())

print(df.to_csv(path_or_buf = "two_stocks.csv")) # writing to local csv
df2 = pd.read_csv("two_stock.csv", header = [0,1], index_col = [0], parse_dates = [0])
print(df2)
print(df2.info())



Equity Analysis with Python / Ticker Object

In [ ]:
aapl = yf.Ticker(ticker = "AAPL")
print(aapl)
print(aapl.history()) # historical price and volume data
print(aapl.get_info())

info = pd.Series(aapl.get_info())
print(info)
print(info.head(50))



Market Capitalization / Shares Outstanding / Share price


In [ ]:
market_cap = info.marketCap
print(market_cap)

shares = info.sharesOutstanding
print(shares)

price = info.currentPrice
print(price)

print(market_cap / shares)


Equity Value, Firm Value and Financail Distress

In [ ]:
market_cap =info.marketCap
print(market_cap)

debt = info.totalDebt
print(debt)

firm_value = market_cap + debt
print(firm_value)

plt.bar(["Firm Value"], [market_cap], label = "Equity", width = 0.5)
plt.bar(["Firm Value"], [debt], label = "Debt", width = 0.5, bottom = market_cap)
plt.xlim(-1,1)
plt.legend(fontsize = 12)
plt.title("Firm Value Apple Inc", fontsize = 15)
plt.show()


Price / Market Value vs Book Value

In [ ]:
price = info.currentPrice
print(price)

bv = info["bookValue"]
print(bv)

price_book = price/bv
print(price_book)

symbol = "GM"
print(yf.Ticker(ticker = symbol).get_info()["priceTobook"])


Balance Sheet / Finanical Statements

In [ ]:
bs = aapl.get_balance_sheet() / 1000000 # annual (in million USD)
print(bs)

inc_stat = aapl.get_income_stmt() / 1000000 # (in million USD)
print(inc_stat)

cf = aapl.get_cashflow() / 1000000 # (in million USD)
print(cf)

print(aapl.quarterly_balance_sheet)
print(aapl.quarterly_income_stmt)
print(aapl.quarterly_cashflow)

Finanical Data Analysis and Performance Evaluation with Python

In [ ]:
df = pd.read_csv("multi_assets.csv", header = [0,1], index_col = 0, parse_dates = [0])
print(df, df.info())

close = df.Close.copy()
print(close)

close.BA.dropna().plot(figsize = (15,8), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

close.describe()
close.dropna().plot(figszie = (15,8), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()



Normalizing Financial Time Series to a Base Value(100)

In [ ]:
close.iloc[0,0]
close.BA.div(close.iloc[0,0]).mul(100)
close.iloc[0]
norm = close.div(close.iloc[0]).mul(100)

norm.dropna().plot(figszie = (15, 8), fontsize = 13, logy = False)
plt.legend(fontsize = 13)
plt.show()

close.to_csv("close.csv")

In [ ]:
start = "2015-01-02"
end = "2020-12-31"
symbol = ["GE", "Reliance.NS"]

df = yf.download(symbol, start, end)
close = df.Close.copy()
print(df, close)

close.GE.dropna().plot(figsize = (15,8), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

norm = close.div(close.iloc[0]).mul(1)
print(norm) 



Price Changes and Financial Returns

In [ ]:
close = pd.read_csv("close.csv", index_col = "Date", parse_dates = ["Date"])
msft = close.MSFT.dropna().to_frame().copy()
print(close, msft)

msft.rename(columns = {"MSFT": "Price"}, inplace = True)
print(msft)

msft.shift(periods = 1)
print(msft)

msft["P_lagl"] = msft.shift(periods = 1)
print(msft)

msft["P_diff"] = msft.Price.sub(msft.P_lagl)
print(msft)

msft.P_diff.equals(msft.P_diff2)
msft.Price.div(msft.P_lagl) - 1
print(msft)

msft["Returns"] = msft.Price.pct_change(periods = 1)
print(msft)
print(46.0900 / 45.7600 - 1)

msft.drop(columns = ["P_lagl", "P_diff", "P_diff2"], inplace = True)
print(msft)

msft.to_csv("msft.csv")
print(msft)

Measuring Reward and Risk of an Investment 

In [ ]:
msft = pd.read_csv("msft.csv", index_col = "Date", parse_dates = ["Date"])
print(msft)

msft.price.plot(figsize = (15,8), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()
print(msft.describe())

mu = msft.Returns.menu()
print(mu)

sigma = msft.Returns.std()
print(sigma, np.sqrt(msft.Returns.var()))


Investment Multiple and CAGR

In [ ]:
multiple = (msft.price[-1] / msft.Price[0])
print(multiple)
print((multiple - 1) * 100)
print(msft.Price / msft.Price[0])

start = msft.index[0]
end = msft.index[-1]
td = end - start
td_years = td.days /365.25
print(start, end, td, td_years)

cagr = (msft.price[-1]) **(1/((msft.index[-1] - msft.index[0].days / 365.15)) - 1)
print(cagr)

print((1 + cagr) **(td_years))

Compound Returns & Geometric Mean Return

In [ ]:
multiple = (1 + msft.Return).prod()
print(multiple)

n = msft.Returns.count()
print(n)

geo_mean = multiple**(1/n) - 1
print(geo_mean)
print((1 + geo_mean)**n)

mu = msft.Returns.mean()
print(mu)

print((1 + mu)**n)

Discrete Compounding

In [ ]:
PV = 100
r = 0.08
n = 1
m = 4
print(100 * 1.08)

FV = PV * (1 + r)**n
print(FV)

effective_annual_rate = (FV/PV)**(1/n) - 1
print(effective_annual_rate)

FV = PV * (1 + r/m)**(n*m)
print(FV)

efferctive_annual_rate = (FV/PV)**(1/n) - 1
print(effective_annual_rate)



Simple Returns vs Log Returns (Part 1)

In [ ]:
df = pd.DataFrame(data = [100,50,90], columns = ["Price"])
print(df)

df["SR"] = df.price.pct_chnage() # simple returns
df["LR"] = np.log(df.Price / df.price.shift()) # Log returns
print(df)

periods = df.SR.count()
print(periods)

mean_sr = df.SR.mean()
print(mean_sr)

100 * (1 + df.SR).prod()**(periods)

geo_mean = (1 + df.SR).prod()**(1/periods) - 1
print(geo_mean)

print(100 * (1 + geo_mean)**periods)

sum_lr = df.LR.sum()
print(sum_lr)

mean_lr = df.LR.mean()
print(mean_lr)

print(100 * np.exp(mean_lr * periods))


Simple Returns vs. Log Returns(Part 2)

In [ ]:
msft = pd.read_csv("msft.csv", index_col = "Date", parse_dates = ["Date"])
print(msft)

msft.Returns.add(1).prod()
np.exp(msft.log_ret.sum())
msft.Returns.add(1).cumprod()
np.exp(msft.log_ret.cumsum())
msft.log_ret.cumsum().apply(np.exp)
(msft.Price[-1]/msft.Price[0])**(1/((msft.index[-1] - msft.index[0]).days / 365.25)) -1

trading_days_year = msft.Returns.count() / ((msft.index[-1] - msft.index[0]).days / 365.25)
print(trading_days_year)

np.exp(msft.log_ret.mean() * trading_days_year) - 1
msft.Returns.mean() * trading_days_year
np.exp(msft.log_ret.mean() * 252) - 1


Price Return vs. Total Return(Equities)

In [ ]:
 df = pd.read_csv("multi_assets.csv", header = [0,1]), index_col = 0, parse_dates = [0])["Adj Close"]
 print(df)

 close["BA_TR"] = df.BA
 close["MSFT_TR"] = df.MSFT

 returns = close.apply(lambda x: np.log(x.dropna() / x.dropna().shift()))
 print(returns)

 summary = returns.agg(["mean", "std"]).T
 print(summary)

 summary.columns = ["Mean", "std"]
 print(summary)

Normality of Financial Returns

In [ ]:
msft["log_ret"] = np.log(msft.Price / msft.Price.shift())
print(msft, msft.describe())

msft.log_ret.plot(kind = "hist", figsize = (15,8), bins = 100, fontsize = 15, density = False)
plt.xlabel("Daily Returns", fontsize = 15)
plt.ylabel("Frequency", fontsize = 15)
plt.title("Frequancy Distribution of Returns", fontsize = 20)
plt.show()

mu = msft.log_ret.mean()
sigma = msft.log_ret.std()
print(mu, sigma)

stats.skew(msft.log_ret.dropna())
stats.kurtosis(msft.log_ret.dropna(), fisher = True)

x = np.linspace(msft.log_ret.min(), msft.log_ret.max(), 10000)
y = stats.norm.odf(x, loc = mu, scala = sigma)
print(x, y)

plt.figure(figsize = (20,8))
plt.hist(msft.log_ret, bins = 500, density = True, label = "Frequency Distribution of daily Returns")
plt.plot(x, y, linewidth = 3, color = "red", label = "Normal Distribution")
plt.title("Normal Distribution", fontsize = 20)
plt.xlabel("Daily Returns", fontsize = 15)
plt.ylabel("pdf", fontsize = 15)
plt.legend(fontsize = 15)
plt.show()



Rolling Statistics

In [ ]:
ann_mu = msft.log_ret.mean() *  252
print(ann_mu)

ann_std = msft.log_ret.std() * np.sqrt(252)
print(ann_std)

window = 252
print(msft.log_ret.rolling(window = 252))
print(msft.log_ret.rolling(window = 252).sum())

roll_std = msft.log_ret.rolling(window = 252).std() * np.sqrt(252)
print(roll_std)

roll_std.plot(figsize = (12,8))
plt.show()

roll_std = msft.log_ret.rolling(window = 252).std() * np.sqrt(252)
print(roll_std)

roll_std.plot(figsize = (12,8))
plt.show()

print(roll_mean.iloc[250:])
roll_mean.plot(figsize = (12,8))
plt.show()

sma_window = 50
msft.Price.plot(figsize = (12,8))
msft.Price.rolling(sma_window).mean().plot()
plt.show()

Short Selling / Short Positions

In [ ]:
close["USDEUR=X"] = 1/close["USDEUR=X"]
print(close)

fx = close[["EURUSD=X", "USDEUR=X"]].dropna().copy()
print(fx)

fx.plot(figsize = (12,8), fontsize = 13)
plt.legend(fontsize = 13)
plt.show()

simple_ret = fx.pct_change()
print(simple_ret)

log_ret = np.log(fx / fx.shift())
print(log_ret)
print(log_ret.sum())

norm_fx = log_ret.cumsum().apply(np.exp)
print(norm_fx)
print(norm_fx.iloc[0] = [1,1], norm_fx)

norm_fx.plot(figsize = (12,8), fontsize =13)
plt.legend(fontsize  = 13)
plt.show()


Covariance and Correlation

In [ ]:
close["USEDEUR=X"] = 1/close["EURUSD=X"]
print(close)

returns = close.apply(lambda x: np.log(x.dropna() / x.dropna().shift()))
print(returns, returns.cov(), returns.corr())

plt.figure(figsize = (12,8))
sns.set(font_scale = 1.4)
sns.heatmap(returns.corr(), cmap = "RdYlBu_r", annot = True, annot_kws = {"size": 15}, vmin = -1, vmax = 1)
plt.show()


Portfolio of Assets and Portfolio Returns

In [ ]:
prices = pd.DataFrame(data = {"Asset_A": [100,112], "Asset_B": [100,104], index = [0,1]})
print(prices)

prices["Total"] = prices.Asset_A + prices.Asset_B
print(prices)

returns = prices.pct_change()
print(returns)

log_returns = np.log(prices / prices.shift())
print(log_returns)

print(0.5 * log_returns.iloc[1,0] + 0.5 * log_returns.iloc[1,1])



Margin Trading & Levered Returns

In [ ]:
P0 = 100
P1 = 110
leverage = 2
margin = P0/2
print(margin)

unlev_return = (P1 - P0) / P0
print(unlev_return)

lev_return = (P1 - P0) / margin
print(lev_return)

unlev_return = np.log((P1 - P0) / P0 + 1)
print(unlev_return)

lev_return = np.log((P1 - P0) / margin + 1)
print(lev_return)

ETF Investing and Index Replication / Tracking

In [ ]:
index = "^GSPC"
index_tr = "^SP500TR"
etf = "SPY"

df = yf.download([index, index_tr, etf], end = "2020-11-30")
print(df)

prices = df.Close.copy()
print(prices)
print(prices["SPYTR"] = df[("ADj Close", "SPY")])

norm = prices / prices.iloc[0]
print(norm)

norm[["SPY", "^GSPC"]].plot(figsize = (12,8))
plt.title("S&P500: ETF Price vs Price Return Index", fontsize = 15)
plt.show()

norm.loc[: ["SPYTR", "^SP500TR"]].plot(figsize = (12,8))
plt.title("S&P500: ETF Tr vs Total Return Index", fontsize = 15)
plt.show()

returns = prices.pct_change()
print(returns)

def ann_risk_return(returns_df):
  summary = pd.DataFrame(index = returns_df.columns)
  summary["ann. Risk"] = returns_df.std() * np.sqrt(252)
  log_returns = np.log(returns_df + 1)
  summary["CAGR"] = np.exp(log_returns.mean() * 252) - 1
  return summary

summary = ann_risk_return(returns)
print(summary)


Active Return and Active Risk (Tracking Error)

In [ ]:
price_ret = returns[["^GSPC", "SPY"]]
print(price_ret)

total_ret = returns[["^SP500TR", "SPYTR"]]
print(total_ret)

def tracking(returns_df, index):
  active_returns = returns_df.sub(returns_df[index], axis = "rows")
  summary = pd.DataFrame(index = returns_df.columns)
  summary["TrackingError"] = active_returns.std() * np.sqrt(252)
  log_returns = np.log(active_returns + 1)
  return summary 

print(tracking(price_ret, "^GSPC"))
print(tracking(total_ret, "^SP500TR"))

Etf Trading with the Interactive Brokers(IBKR) API

In [ ]:
from ib_insync import * 
util.startLoop()

ib = IB()
ib.connect()
symbol = "SPY"

contract = Stock[symbol, "SMART", "USD"]
print(contract)

cd = ib.reqContractDetails(contract)
print(cd, len(cd))
print(ib.qualityContracts(contract))

data = ib.reqMktData(contract)
print(data)
print(ib.reqMarketDataType(3))
print(data.marketPrice())

order = MarketOrder(action = "BUY", totalQuantity = 1)
print(order)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
  ib.waitOnUpdate()

print(trade.orderStatus.status)
print(trade.orderStatus.avgFillPrice)

pos = ib.positions()
print(pos)

df = util.df(pos)
print(df)

df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
print(df)

ib.disconnect()